In [1]:
!pip install -U vnstock

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.9/120.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 8.9 MB/s eta 0:00:00


In [2]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=80c40a20e26e7ec0936cebb19afb45b1501a6a6cade0c4d0870403a0cb75e583
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [3]:
pip install tensorflow

In [4]:
!pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=43a253854e9b5f42389e5840225345ff825bf4024f84e26cd4ab9f6bce380f86
  Stored in directory: /root/.cache/pip/wheels/46/f9/96/709295c836133071c12a300729fed4027757f889c01695feea
Successfully built keras-self-attention


In [5]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.5 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
from ta.momentum import RSIIndicator
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, RNN, concatenate
from tensorflow.keras.optimizers import RMSprop
from vnstock import *
from ta.volatility import BollingerBands
from tensorflow.keras.layers import Layer
import tensorflow as tf
from keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, Flatten, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier

from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import CCIIndicator
from sklearn.preprocessing import StandardScaler
from ta.volume import ChaikinMoneyFlowIndicator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import regularizers
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.saving import register_keras_serializable
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [32]:
from vnstock import *
import pandas as pd
import os
from collections import Counter
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
from sklearn.metrics import accuracy_score # Import accuracy_score

In [36]:
pip install xgboost

In [37]:
from xgboost import XGBClassifier

In [8]:
def load_stock(symbol, folder_path='stock_data'):
    """
    Load dữ liệu cổ phiếu từ mã cổ phiếu, sử dụng đường dẫn thư mục cố định.

    Parameters:
        symbol (str): Mã cổ phiếu (ví dụ: 'VCB')
        folder_path (str): Thư mục chứa file Excel (mặc định là 'stock_data')

    Returns:
        DataFrame chứa dữ liệu cổ phiếu.
    """
    file_path = os.path.join(folder_path, f"{symbol}.xlsx")

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ Không tìm thấy file: {file_path}")

    try:
        df = pd.read_excel(file_path)
        print(f"✅ Đã load dữ liệu cho mã: {symbol}")
        return df
    except Exception as e:
        print(f"❌ Lỗi khi load dữ liệu {symbol}: {e}")
        return pd.DataFrame()

In [10]:
df_vhm = load_stock('VNM','/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA/v1/train_data')

✅ Đã load dữ liệu cho mã: VNM


In [11]:
df_vhm

,time,open,high,low,close,volume,ticket
0,2019-01-02,77.47,79.07,77.15,79.07,403570,VNM
1,2019-01-03,79.07,79.07,77.54,78.37,449730,VNM
2,2019-01-04,78.11,79.70,77.15,79.70,498470,VNM
3,2019-01-07,81.43,83.53,80.02,83.53,897430,VNM
4,2019-01-08,83.79,83.79,82.06,82.89,448350,VNM
...,...,...,...,...,...,...,...
1013,2023-01-19,73.47,74.11,72.75,73.93,2628489,VNM
1014,2023-01-27,74.02,74.47,72.38,72.75,2333482,VNM
1015,2023-01-30,72.75,72.75,72.20,72.20,1614712,VNM
1016,2023-01-31,72.02,72.29,69.93,70.11,4819134,VNM


In [12]:
def load_and_process_data(symbol, drop_na=True, file_path = ''):
    data = load_stock(symbol,file_path)
    if data.empty:
        return None

    df = data

     # Tính toán khối lượng trung bình
    volume_ma_period = 20  # Ví dụ: khối lượng trung bình 20 ngày
    df['volume_ma'] = df['volume'].rolling(window=volume_ma_period).mean()


    # Tính tỷ lệ khối lượng hiện tại / khối lượng trung bình
    df['volume_to_volume_ma_ratio'] = df['volume'] / df['volume_ma']


    # Tính toán EMA ngắn hạn (12 ngày)
    df['ema_12'] = df['close'].ewm(span=12, adjust=False).mean()


    # Tính toán EMA dài hạn (26 ngày)
    df['ema_26'] = df['close'].ewm(span=26, adjust=False).mean()

    # Tính toán SMA trung hạn (20 ngày)
    df['sma_20'] = df['close'].rolling(window=20).mean()

    # Tính toán SMA dài hạn (50 ngày)
    df['sma_50'] = df['close'].rolling(window=50).mean()

     # Tính toán ROC

    # df['roc_1'] = ((df['close'] - df['close'].shift(1)) / df['close'].shift(1)) * 100
    df['roc_5'] = ((df['close'] - df['close'].shift(5)) / df['close'].shift(5)) * 100
    # df['roc_9'] = ((df['close'] - df['close'].shift(9)) / df['close'].shift(9)) * 100

    # Tính toán %K (Stochastic Oscillator)
    stoch_period = 14
    df['%K'] = ((df['close'] - df['low'].rolling(window=stoch_period).min()) /
                (df['high'].rolling(window=stoch_period).max() - df['low'].rolling(window=stoch_period).min())) * 100

    # Tính toán %R (Williams %R)
    df['%R'] = ((df['high'].rolling(window=stoch_period).max() - df['close']) /
                (df['high'].rolling(window=stoch_period).max() - df['low'].rolling(window=stoch_period).min())) * -100

    # Tính toán CCI (Commodity Channel Index)
    cci_period = 20
    df['typical_price'] = (df['high'] + df['low'] + df['close']) / 3
    df['cci'] = CCIIndicator(high=df['high'], low=df['low'], close=df['close'], window=cci_period).cci()

    # Tính toán OBV (On Balance Volume)
    df['obv'] = OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()


    # Tính toán MACD
    df['macd'] = df['ema_12'] - df['ema_26']
    # Tính toán Signal line (EMA 9 ngày của MACD)
    df['signal_line'] = df['macd'].ewm(span=9, adjust=False).mean()
    # Tính toán MACD histogram
    df['macd_histogram'] = df['macd'] - df['signal_line']


    # Tính toán RSI
    rsi_period = 14
    df['rsi'] = RSIIndicator(df['close'], window=rsi_period).rsi()


    # Bollinger Bands
    indicator_bb = BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_bbm'] = indicator_bb.bollinger_mavg()
    df['bb_bbh'] = indicator_bb.bollinger_hband()
    df['bb_bbl'] = indicator_bb.bollinger_lband()
    df['bb_bbp'] = indicator_bb.bollinger_pband()

    df['bb_bbh_bb_bbl_ratio'] = df['bb_bbh'] / df['bb_bbl']


    # Xứ lý với VNINDEX
    data_time = data['time']

    data_vnd = load_stock('VNINDEX', file_path)
    data_vnd = data_vnd.drop(columns=['ticket'])
    data_vnd.columns = ['time','open_vnd', 'high_vnd', 'low_vnd', 'close_vnd', 'volume_vnd']


    # Tính toán RSI
    rsi_period = 14
    data_vnd['rsi_vnd'] = RSIIndicator(data_vnd['close_vnd'], window=rsi_period).rsi()
    # Tính toán RSI-base-MA
    ma_period = 9
    data_vnd['rsi_base_ma_vnd'] = data_vnd['rsi_vnd'].rolling(window=ma_period).mean()
    data_vnd['rsi_rsi_base_ma_ratio_vnd'] = data_vnd['rsi_vnd'] / data_vnd['rsi_base_ma_vnd']

    # Tính toán khối lượng trung bình
    volume_ma_period = 20  # Ví dụ: khối lượng trung bình 20 ngày
    data_vnd['volume_ma_vnd'] = data_vnd['volume_vnd'].rolling(window=volume_ma_period).mean()
    # Tính tỷ lệ khối lượng hiện tại / khối lượng trung bình
    data_vnd['volume_to_volume_ma_ratio_vnd'] = data_vnd['volume_vnd'] / data_vnd['volume_ma_vnd']

    # Bollinger Bands
    indicator_bb = BollingerBands(close=data_vnd['close_vnd'], window=20, window_dev=2)
    data_vnd['bb_bbm_vnd'] = indicator_bb.bollinger_mavg()
    data_vnd['bb_bbh_vnd'] = indicator_bb.bollinger_hband()
    data_vnd['bb_bbl_vnd'] = indicator_bb.bollinger_lband()
    data_vnd['bb_bbp_vnd'] = indicator_bb.bollinger_pband()

    data_vnd['bb_bbh_bb_bbl_ratio_vnd'] = data_vnd['bb_bbh_vnd'] / data_vnd['bb_bbl_vnd']

    # Tính toán ROC
    roc_period = 9  # Chu kỳ 9 ngày, bạn có thể thay đổi tùy ý
    data_vnd['roc_vnd'] = ((data_vnd['close_vnd'] - data_vnd['close_vnd'].shift(roc_period)) / data_vnd['close_vnd'].shift(roc_period)) * 100

    # Tính toán %K (Stochastic Oscillator)
    stoch_period = 14
    data_vnd['%K_vnd'] = ((data_vnd['close_vnd'] - data_vnd['low_vnd'].rolling(window=stoch_period).min()) /
                (data_vnd['high_vnd'].rolling(window=stoch_period).max() - data_vnd['low_vnd'].rolling(window=stoch_period).min())) * 100

    # Tính toán %R (Williams %R)
    data_vnd['%R_vnd'] = ((data_vnd['high_vnd'].rolling(window=stoch_period).max() - data_vnd['close_vnd']) /
                (data_vnd['high_vnd'].rolling(window=stoch_period).max() - data_vnd['low_vnd'].rolling(window=stoch_period).min())) * -100

    # Tính toán CCI (Commodity Channel Index)
    cci_period = 20
    data_vnd['typical_price_vnd'] = (data_vnd['high_vnd'] + data_vnd['low_vnd'] + data_vnd['close_vnd']) / 3
    data_vnd['cci_vnd'] = CCIIndicator(high=data_vnd['high_vnd'], low=data_vnd['low_vnd'], close=data_vnd['close_vnd'], window=cci_period).cci()

    # Tính toán OBV (On Balance Volume)
    data_vnd['obv_vnd'] = OnBalanceVolumeIndicator(close=data_vnd['close_vnd'], volume=data_vnd['volume_vnd']).on_balance_volume()

    # Tính toán EMA ngắn hạn (12 ngày)
    data_vnd['ema_12_vnd'] = data_vnd['close_vnd'].ewm(span=12, adjust=False).mean()

    # Tính toán EMA dài hạn (26 ngày)
    data_vnd['ema_26_vnd'] = data_vnd['close_vnd'].ewm(span=26, adjust=False).mean()

    # Tính toán SMA trung hạn (20 ngày)
    data_vnd['sma_20_vnd'] = data_vnd['close_vnd'].rolling(window=20).mean()

    # Tính toán SMA dài hạn (50 ngày)
    data_vnd['sma_50_vnd'] = data_vnd['close_vnd'].rolling(window=50).mean()

    data_vnd['time'] = pd.to_datetime(data_vnd['time']).dt.date
    # merge
    df['time'] = pd.to_datetime(df['time']).dt.date
    df = pd.merge(df, data_vnd, on='time')


     # Tính tỷ lệ thay đổi giá
    threshold=0.01
    df['change'] = (df['close'].shift(-1) - df['close']) / df['close']

    # Gán nhãn
    df['target'] = 0  # Hold mặc định
    df.loc[df['change'] > threshold, 'target'] = 1   # Buy
    df.loc[df['change'] < -threshold, 'target'] = 2  # Sell

    if drop_na:
        df.dropna(inplace=True)

    return df

In [13]:
df_vhm_process = load_and_process_data("VHM", file_path= '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA/v1/train_data')

✅ Đã load dữ liệu cho mã: VHM
✅ Đã load dữ liệu cho mã: VNINDEX


In [14]:
df_vhm_process

,time,open,high,low,close,volume,ticket,volume_ma,volume_to_volume_ma_ratio,ema_12,...,%R_vnd,typical_price_vnd,cci_vnd,obv_vnd,ema_12_vnd,ema_26_vnd,sma_20_vnd,sma_50_vnd,change,target
49,2019-03-19,69.17,69.31,68.08,68.44,1100900,VHM,1047159.50,1.051320,67.416190,...,-16.150082,1008.440000,117.650607,1911916753,997.995388,980.523935,992.3235,944.8256,-0.019141,2
50,2019-03-20,67.79,68.00,66.26,67.13,1061950,VHM,1059478.00,1.002333,67.372161,...,-27.768933,999.190000,43.505368,1731871123,998.657636,982.136977,993.9095,947.0366,-0.043349,2
51,2019-03-21,67.71,67.71,64.22,64.22,1009930,VHM,1049481.00,0.962314,66.887213,...,-98.673500,989.976667,-34.242716,1549163093,996.061077,982.110534,993.6200,949.1078,0.032856,1
52,2019-03-22,64.80,66.40,64.07,66.33,957080,VHM,1049787.50,0.911689,66.801488,...,-77.687444,986.693333,-60.237723,1700974173,994.930142,982.599383,993.6100,951.2640,-0.023066,2
53,2019-03-25,65.16,65.16,63.71,64.80,793070,VHM,1047539.00,0.757079,66.493567,...,-89.326633,970.453333,-164.581097,1523341503,991.105504,981.671281,992.3920,952.8726,0.010185,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,2023-01-18,52.00,52.90,50.90,52.50,1265675,VHM,1484643.60,0.852511,50.921160,...,-2.261522,1096.076667,157.861811,50953764568,1062.052982,1047.166167,1040.7145,1020.0316,-0.005714,0
1013,2023-01-19,52.40,53.40,51.50,52.20,2236566,VHM,1502375.85,1.488686,51.117904,...,-3.478926,1104.753333,161.374773,51571478904,1069.134062,1051.678303,1045.1745,1022.4814,0.021073,1
1014,2023-01-27,52.40,53.90,51.90,53.30,1283501,VHM,1457968.85,0.880335,51.453611,...,-6.376196,1117.603333,173.107664,52176412304,1076.513437,1056.524355,1049.8990,1025.8786,-0.033771,2
1015,2023-01-30,53.30,53.50,51.50,51.50,1350577,VHM,1458064.10,0.926281,51.460748,...,-26.262993,1107.866667,135.555465,51377736604,1080.522139,1059.935143,1054.0105,1028.8394,-0.011650,2


In [16]:
#  Hàm xử lý và chuẩn hóa dữ liệu
scaler = None
def process_and_create_data(symbol, features, target, time_steps=5, scaler=None, drop_na=True, file_path = '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA/v1/train_data'):
    df = load_and_process_data(symbol, drop_na, file_path= file_path)
    if df is None or len(df) < time_steps:
        return None, None  # Trả về None nếu dữ liệu không đủ dài
    if scaler is None:
        # scaler = RobustScaler()
        scaler = StandardScaler()
        # scaler = MinMaxScaler()
        df[features] = scaler.fit_transform(df[features])
    else:
        df[features] = scaler.transform(df[features])

    X = df[features]
    y = df[target]
    return X, y, scaler

In [17]:
# Xử lý và kết hợp dữ liệu cho nhiều cổ phiếu để tạo tập dữ liệu
# vn30_symbols = ["ACB","BCM","BID","BVH","CTG","FPT","GAS","GVR","HDB","HPG","MBB","MSN","MWG","PLX","POW","SAB","SHB","SSB","SSI","STB","TCB","TPB","VCB","VHM","VIB","VIC","VJC","VNM","VPB","VRE"]
vn30_symbols = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']

# Đặc trưng và biến mục tiêu
features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','rsi_vnd','rsi_base_ma_vnd','rsi_rsi_base_ma_ratio_vnd','volume_ma_vnd','volume_to_volume_ma_ratio_vnd','bb_bbm_vnd','bb_bbh_vnd','bb_bbl_vnd','bb_bbp_vnd','bb_bbh_bb_bbl_ratio_vnd','roc_vnd','%K_vnd','%R_vnd','cci_vnd','obv_vnd','ema_12_vnd','ema_26_vnd','sma_20_vnd','sma_50_vnd']
target = 'target'

# Khởi tạo danh sách để lưu dữ liệu
X_all, y_all = [], []
scaler = None

for symbol in vn30_symbols:
    X, y, scaler = process_and_create_data(symbol, features, target,5, scaler, file_path = '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA/v1/train_data')
    if X is not None and y is not None:
        X_all.append(X)
        y_all.append(y)

# Kết hợp tất cả dữ liệu lại
X_all = np.concatenate(X_all, axis=0)
y_all = np.concatenate(y_all, axis=0)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

✅ Đã load dữ liệu cho mã: ACB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BCM
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BID
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BVH
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: CTG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: FPT
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: GAS
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: GVR
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: HDB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: HPG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: LPB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MBB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MSN
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MWG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: PLX
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: SAB
✅ Đã load 

In [18]:
X_train.shape

(22758, 39)

In [19]:
counts = Counter(y_train)

print(f"Số lượng 2 (Sell): {counts[2]}")
print(f"Số lượng  0 (Hold): {counts[0]}")
print(f"Số lượng  1 (Buy):  {counts[1]}")

Số lượng 2 (Sell): 5577
Số lượng  0 (Hold): 11188
Số lượng  1 (Buy):  5993


In [ ]:
# =========================================Sử dụng SMOTE để cân bằng lớp [Option]===========================================

In [ ]:
# Ví dụ: chỉ cân bằng lớp 2 (Sell) và lớp 1 (Buy) lên mức 5000 mẫu
smote = SMOTE(sampling_strategy={1: 5000, 2: 5000}, random_state=42)

In [ ]:
# Chuyển về 2D
X_train_reshaped = X_train.reshape((X_train.shape[0], -1))



In [ ]:
# Áp dụng SMOTE
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_reshaped, y_train)

In [ ]:
# Sau đó reshape lại để đưa vào mô hình (nếu cần)
X_train_resampled = X_train_resampled.reshape((-1, 5, len(features)))

In [ ]:
# In lại phân phối nhãn sau khi cân bằng
print("Phân phối sau SMOTE:", Counter(y_train_resampled))

Phân phối sau SMOTE: Counter({0: 16709, 1: 5000, 2: 5000})


In [ ]:
# =========================================Sử dụng SMOTE để cân bằng lớp [Option]===========================================

In [20]:
# Load dữ liệu test
# Xử lý và kết hợp dữ liệu cho nhiều cổ phiếu để tạo tập dữ liệu
# vn30_symbols = ["ACB","BCM","BID","BVH","CTG","FPT","GAS","GVR","HDB","HPG","MBB","MSN","MWG","PLX","POW","SAB","SHB","SSB","SSI","STB","TCB","TPB","VCB","VHM","VIB","VIC","VJC","VNM","VPB","VRE"]
vn30_symbols = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']

# Đặc trưng và biến mục tiêu
features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','rsi_vnd','rsi_base_ma_vnd','rsi_rsi_base_ma_ratio_vnd','volume_ma_vnd','volume_to_volume_ma_ratio_vnd','bb_bbm_vnd','bb_bbh_vnd','bb_bbl_vnd','bb_bbp_vnd','bb_bbh_bb_bbl_ratio_vnd','roc_vnd','%K_vnd','%R_vnd','cci_vnd','obv_vnd','ema_12_vnd','ema_26_vnd','sma_20_vnd','sma_50_vnd']
target = 'target'

# Khởi tạo danh sách để lưu dữ liệu
X_all, y_all = [], []
# scaler = None

for symbol in vn30_symbols:
    X, y, scaler = process_and_create_data(symbol, features, target,5, scaler, file_path = '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA/v1/test_data')
    if X is not None and y is not None:
        X_all.append(X)
        y_all.append(y)

# Kết hợp tất cả dữ liệu lại
X_test = np.concatenate(X_all, axis=0)
y_test = np.concatenate(y_all, axis=0)



✅ Đã load dữ liệu cho mã: ACB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BCM
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BID
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BVH
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: CTG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: FPT
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: GAS
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: GVR
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: HDB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: HPG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: LPB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MBB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MSN
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MWG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: PLX
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: SAB
✅ Đã load 

In [21]:
X_test.shape

(6000, 39)

In [22]:
counts = Counter(y_test)

print(f"Số lượng 2 (Sell): {counts[2]}")
print(f"Số lượng  0 (Hold): {counts[0]}")
print(f"Số lượng  1 (Buy):  {counts[1]}")

Số lượng 2 (Sell): 1049
Số lượng  0 (Hold): 3814
Số lượng  1 (Buy):  1137


In [24]:
# Tính trọng số cân bằng
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

print("Class Weights:", class_weight_dict)

Class Weights: {np.int64(0): np.float64(0.6780479084733643), np.int64(1): np.float64(1.2658101117970966), np.int64(2): np.float64(1.360229514075668)}


In [27]:
# Dùng class_weight
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)

In [28]:
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight={np.int64(0): np.float64(0.6780479084733643),
                                     np.int64(1): np.float64(1.2658101117970966),
                                     np.int64(2): np.float64(1.360229514075668)},
                       random_state=42)

In [30]:
y_pred_rf = rf.predict(X_test)

In [33]:
print("Accuracy:", accuracy_score(y_test, y_pred_rf))

Accuracy: 0.6096666666666667


In [34]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

Confusion Matrix:
[[3501  272   41]
 [ 972  139   26]
 [ 919  112   18]]


In [35]:
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.92      0.76      3814
           1       0.27      0.12      0.17      1137
           2       0.21      0.02      0.03      1049

    accuracy                           0.61      6000
   macro avg       0.38      0.35      0.32      6000
weighted avg       0.50      0.61      0.52      6000



In [38]:
model_xgb = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',  # dùng cho đa lớp
    random_state=42,
    class_weight=class_weight_dict
)


In [39]:
model_xgb.fit(X_train, y_train)

[08:46:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "class_weight", "use_label_encoder" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight={np.int64(0): np.float64(0.6780479084733643),
                            np.int64(1): np.float64(1.2658101117970966),
                            np.int64(2): np.float64(1.360229514075668)},
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mloglo...
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [40]:
y_pred_xgb = model_xgb.predict(X_test)

In [41]:
print("🎯 Accuracy:", accuracy_score(y_test, y_pred_xgb))

🎯 Accuracy: 0.6171666666666666


In [42]:
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_xgb))


📊 Confusion Matrix:
[[3513  273   28]
 [ 948  179   10]
 [ 925  113   11]]


In [43]:
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred_xgb))


📋 Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.92      0.76      3814
           1       0.32      0.16      0.21      1137
           2       0.22      0.01      0.02      1049

    accuracy                           0.62      6000
   macro avg       0.40      0.36      0.33      6000
weighted avg       0.51      0.62      0.53      6000

